In [2]:
import numpy as np

In [36]:
npy_path = "/Users/sherryyang/Documents/wiggins-lab/data/0820/xy0/phase/211121_37C_10_1176_HU_timelapse002_prob211203_HU_t00010xy0c1_seg.npy"
dat = np.load(npy_path, allow_pickle=True).item()
gradientsx = np.gradient(dat['flows'][-1][0])
gradientsy = np.gradient(dat['flows'][-1][1])
gradient_total = gradientsy[1] + gradientsx[0]
regs_label = dat['masks']

In [37]:
import pandas as pd

label_info_df = pd.DataFrame(columns = ['label', 'x_mean', 'y_mean', 'area'])

for i in range(1,np.max(regs_label)+1):
    row_indices, col_indices = np.where(regs_label == i)
    if len(row_indices) > 0:
        new_row = {'label': i, 'x_mean': np.average(row_indices), 'y_mean': np.average(col_indices), 'area': len(row_indices)}
        label_info_df.loc[len(label_info_df)] = new_row

In [38]:
label_info_df.to_csv('t00010.csv', index=False)

In [23]:
import json

# Save to JSON file
with open('t11.json', 'w') as file:
    json.dump(label_info_dict, file)


In [ ]:
# Load from JSON file
with open('t10.json', 'r') as file:
    loaded_data = json.load(file)

print(loaded_data)

In [3]:
from scipy.io import loadmat

mat_path = "/Users/sherryyang/Documents/wiggins-lab/data/0820/xy0/seg/211121_37C_10_1176_HU_timelapse002_prob211203_HU_t00008xy0_err.mat"
# Load the MAT file
data = loadmat(mat_path)

# Access a specific variable from the MAT file
variable_name = data['variable_name']

# Now you can work with the variable as a NumPy array
print(variable_name)


KeyError: 'variable_name'

In [24]:
label = data['regs']['regs_label'][0][0]
track_result = data['regs']['map'][0][0]['f'][0][0][0]

In [39]:
superSegger_final_tracker = {}
for i in range(len(track_result)):
    superSegger_final_tracker[i+1] = set(track_result[i][0])

In [44]:
import pandas as pd
from scipy.io import savemat
import numpy as np
import glob
import os
import re

In [46]:
def make_cell_info_reference(labels_mask):
    
    label_info_df = pd.DataFrame(columns = ['label', 'x_mean', 'y_mean', 'area'])
    regs_label = labels_mask

    for i in range(1,np.max(regs_label)+1):
        row_indices, col_indices = np.where(regs_label == i)
        if len(row_indices) > 0:
            new_row = {'label': i, 'x_mean': np.average(col_indices), 'y_mean': np.average(row_indices), 'area': len(row_indices)}
            label_info_df.loc[len(label_info_df)] = new_row

    return label_info_df

In [50]:

npzFiles = glob.glob("/Users/sherryyang/Documents/wiggins-lab/data/0820/xy0/seg/*err.mat")

cell_info_dict = {}
supperSegger_dict = {}

for f in npzFiles:
    pattern = r'_t(\d+)'
    match = re.search(pattern, f)

    if match:
        t_value = match.group(1)
        frame_index = 't' + t_value
    else:
        raise ValueError('No time index on filename.')
    
    data = loadmat(f)
    label = data['regs']['regs_label'][0][0]
    track_result = data['regs']['map'][0][0]['f'][0][0][0]

    superSegger_final_tracker = {}
    for i in range(len(track_result)):
        superSegger_final_tracker[i+1] = set(track_result[i][0])

    cell_info_dict[frame_index] = make_cell_info_reference(label)
    supperSegger_dict[frame_index] = superSegger_final_tracker
